In [1]:
# Author: Patrik Wikstrom (patrik.wikstrom@qut.edu.au)
# Organisation: QUT Digital Media Research Centre 2017
# Version: 0.2

In [42]:
# the_item = "catchoftheday"
the_item = "maisonmargiela"
max_scrape = 150

insta_path = "explore/tags/"
prefix = "photos"

In [43]:
import base64
from string import ascii_letters, digits, punctuation, printable
from IPython.display import Image, display, HTML
from time import sleep
from collections import Counter
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from copy import copy
import requests
import json
import os
import platform
from datetime import datetime
import numpy as np
import pandas as pd

In [44]:
# Create folders as necessary
data_path = "data/data"  + "_" + prefix + "_" + the_item
media_path = "data/media"

print(data_path)
print(media_path)

try:
    os.mkdir("data")
    print("made new data folder")
except:
    pass

try:
    os.mkdir("data/media")
    print("made new media folder")
except:
    pass

try:
    os.mkdir(data_path)
    print("made new project folder")
except:
    pass

data/data_photos_maisonmargiela
data/media


# Load

In [45]:
try:
    f = open(data_path+"/metadata.json","r")
    the_posts = json.load(f)
    f.close()
    print("found",len(the_posts),"saved posts.")
except:
    the_posts = {}
    print("This is a new scrape. No saved posts found.")
    

This is a new scrape. No saved posts found.


In [53]:
try:
    f = open("config.json","r")
except:
    f = open("default-config.json","r")

conf = json.load(f)
f.close()
if not the_item in conf:
    conf[the_item] = conf["default"]
    f = open("config.json","w")
    json.dump(conf,f)
    f.close()
    print("Updated the config file.")
deets = conf[the_item]


# Set up the web driver

In [54]:
if not os.getcwd() in os.get_exec_path():
    print('adding path')
    if platform.system() == "Windows":
        os.environ["PATH"] = os.environ["PATH"] + ";" + os.getcwd()
    else:
        os.environ["PATH"] = os.environ["PATH"] + ":" + os.getcwd()  

In [55]:
driver = webdriver.Firefox()

In [56]:
driver.get("https://www.instagram.com/"+insta_path+the_item+"/")

In [57]:
print(driver.title)

#maisonmargiela • Instagram photos and videos


# Send key strokes to scroll down the page

In [58]:
# first of all click on the "Load more" button

# Instagram uses different class names for the "Load more" button, so it's necessary to check for more than one.
load_more_codes = ["_8imhp","_oidfu"]

found_it = False

for load_more_code in load_more_codes:
    if not found_it:
        try:
            some_element = driver.find_element_by_class_name(load_more_code)
            some_element.click()
            found_it = True
            print(load_more_code)
        except:
            pass

if not found_it:
    print("there is only one page")
    some_element = None

there is only one page


In [60]:
# then press space over an over again for a while
last_how_many = -1
how_many = 0
overlap = set()
all_links = []

# why nine? there are 9 recurring top posts before you get into the recent posts
while last_how_many<how_many and how_many<max_scrape and len(overlap)<=9:
    
    # send a key stroke to the page to scroll down
    some_element = driver.find_element_by_tag_name("body")
    last_how_many = how_many
    for i in range(3):
        some_element.send_keys(" ")
        sleep(0.5)
        
    # check if the posts on the page have already been scraped
#     all_links = [u.get_attribute("href").split("/")[4] for u in driver.find_elements_by_class_name("_mck9w._gvoze._f2mse")]
    all_links = [u.find_elements_by_xpath("./*")[0].get_attribute("href").split("/")[4] for u in driver.find_elements_by_class_name("_mck9w._gvoze._f2mse")]
    overlap = set(the_posts.keys()).intersection(set(all_links))

    how_many = len(driver.find_elements_by_class_name("_mck9w._gvoze._f2mse"))
    print(how_many,end=" | ",flush=True)
print("overlap:",len(overlap))

36 | 48 | 69 | 81 | 93 | 105 | 117 | 129 | 141 | 153 | overlap: 0


In [61]:
# print the number of links/codes found in the page and save the list in a temporary file in case everything breaks down.
print(len(all_links))
f = open(data_path+"/temp_links.json","w")
json.dump(all_links,f)
f.close()

144


# scrape post pages + download media objects (photos or videos)

In [62]:
def fixit(ss):
    return "".join([s for s in ss if s in ascii_letters+digits]).lower()

In [63]:
def clstr(ss):
    return "".join([s for s in ss if s in printable])

In [64]:
def checkup(a_list,ss):
    found_it = False
    for a in a_list:
        if a in ss:
            found_it = True
    return found_it

In [69]:
def extract_hashtags(ss):
    ss = ss.replace("\n"," ")
    plist = ss.split(" ")
    tags = list(p for p in plist if len(p)>0 and p[0]=="#")
    tt = []
    for t in tags:
        tt += t.split("#")
    return list(set(list(fixit(t) for t in tt if len(t)>0)))
        

In [70]:
def download_object(code):
    if not os.path.isfile(media_path+"/"+the_posts[code].get("the_fname","nofile.ppp")):
        try:
            a_media_object = requests.get(the_posts[code]["object_url"],timeout=2)
            with open(media_path+"/"+the_posts[code]["the_fname"], "wb") as f:
                f.write(a_media_object.content)
            print(".",end="",flush=True)
        except:
            print("x",end="",flush=True)
    else:
        print("_",end="",flush=True)


In [71]:
# scrape new post pages + download media objects (photos or videos)
def scrape_single_code(a_code,sleeper=1):
    from ast import literal_eval
    good_stuff = None
    all_fine = True
    try:
        tt = requests.get("https://www.instagram.com/p/"+a_code,timeout=2)
        tt_soup = BeautifulSoup(tt.text, "lxml")
        sleeper = 1
    except:
        all_fine = False
        print("$",end="",flush=True)
        sleep(sleeper)
    
    if all_fine:
        # find all the javascript tags in the code
        all_texts = [u.get_text() for u in tt_soup.find_all("script")]

        # extract the one that starts with this particular string
        all_texts = [at for at in all_texts if "window._sharedData" in at[:50]][0]

        # fix the JavaScript syntax so that it can be transformed to Python code
        all_texts = all_texts.replace("window._sharedData","")
        all_texts = all_texts.replace("true","True")
        all_texts = all_texts.replace("false","False")
        all_texts = all_texts.replace("null","None")
        

        # extract the dict from the code and reach for the good stuff
        stuff = literal_eval(all_texts[3:-1])
        try:
            good_stuff = copy(stuff["entry_data"]["PostPage"][0]["graphql"]["shortcode_media"])
        except:
            return None
        
        # add some extra to the dict
        good_stuff["scrape_ts"] = str(datetime.now())[:19]
        good_stuff["caption_tags"] = extract_hashtags(good_stuff.get("caption",""))
        
        
        if good_stuff["is_video"]:
            good_stuff["object_url"] = good_stuff["video_url"]
            good_stuff["the_fname"] = a_code + ".mp4"
        else:
            good_stuff["object_url"] = good_stuff["display_resources"][-1]['src']
            good_stuff["the_fname"] = a_code + ".jpg"

        print(".",end="",flush=True)
    return good_stuff

In [84]:
# scrape post pages for many codes
def scrape_codes(some_codes):
    new_stuff = {}
    sleeptime=1
    cc = 0
    for a_code in some_codes:
        cc += 1
        if cc % 50==0: print(len(new_stuff))
        new_stuff[a_code] = copy(scrape_single_code(a_code,sleeptime))
        if new_stuff[a_code]==None:
            del new_stuff[a_code]
            sleeptime += sleeptime
        else:
            sleeptime = 10
    return copy(new_stuff)

In [88]:
# add the new stuff to the posts storage
def update_post_storage(new_stuff):
    cc = 0
    for code in new_stuff:
        cc += 1
        if cc % 50 == 0: print()
        if not code in the_posts:
            the_posts[code] = copy(new_stuff[code])
        if not "activity" in the_posts[code]:
            the_posts[code]["activity"] = []
        
        tss = [ts["scrape_ts"] for ts in the_posts[code]["activity"]]
    
        if not new_stuff[code]["scrape_ts"] in tss:
            the_posts[code]["activity"] += [{"scrape_ts":new_stuff[code]["scrape_ts"],
                                         "comments":new_stuff[code]["edge_media_to_comment"],
                                         "likes":new_stuff[code]["edge_media_preview_like"],
                                         "video_views":new_stuff[code].get("video_views","not_a_video")}]
            print(len(the_posts[code]["activity"]),end="",flush=True)
        else:
            # update everything but keep the activity record intact
#            for u in new_stuff[code]:
#                if u != 'activity':
#                    the_posts[code][u] = new_stuff[code][u]
            print("-",end="",flush=True)
    print("\nPosts in storage:",len(the_posts))        

In [86]:
# scrape new post pages (not the photo itself) and add to storage
ns = scrape_codes([u for u in all_links if not u in the_posts])
print("\n"+str(len(ns)))


................................................48
..................................................98
...........................................$.
142


KeyError: 'comments'

In [89]:
update_post_storage(ns)

1111111111111111111111111111111111111111111111111
11111111111111111111111111111111111111111111111111
1111111111111111111111111111111111111111111
Posts in storage: 142


In [90]:
the_posts["BcCgqQWg_fR"]


{'__typename': 'GraphImage',
 'activity': [{'comments': {'count': 0,
    'edges': [],
    'page_info': {'end_cursor': None, 'has_next_page': False}},
   'likes': {'count': 33,
    'edges': [{'node': {'id': '532332428',
       'profile_pic_url': 'https://scontent-lga3-1.cdninstagram.com/t51.2885-19/s150x150/16464547_1728674983824878_2929309378253160448_a.jpg',
       'username': 'litdiesel'}},
     {'node': {'id': '28739430',
       'profile_pic_url': 'https://scontent-lga3-1.cdninstagram.com/t51.2885-19/s150x150/24177785_547952192264041_7578952999087636480_n.jpg',
       'username': 'lucaszach2017'}},
     {'node': {'id': '185516426',
       'profile_pic_url': 'https://scontent-lga3-1.cdninstagram.com/t51.2885-19/s150x150/15802297_215461365580242_3610073806476935168_a.jpg',
       'username': '9.13.80'}},
     {'node': {'id': '5630298362',
       'profile_pic_url': 'https://scontent-lga3-1.cdninstagram.com/t51.2885-19/s150x150/24126278_147560436005310_5267357572670685184_n.jpg',
      

In [78]:
# update metadata for posts in storage

update_frequency = 24 # hours

codes_ready_for_update = []
for code in the_posts:
    latest_scrape = the_posts[code]["activity"][-1]["scrape_ts"]
    time_since_latest_scrape = (datetime.now()-datetime.strptime(latest_scrape,"%Y-%m-%d %H:%M:%S")).total_seconds()
    if time_since_latest_scrape > update_frequency * 3600:
        codes_ready_for_update += [code]

if len(codes_ready_for_update)>0 and input("Do you want to update "+str(len(codes_ready_for_update))+" posts? [Y/N] ") in "Yy":
    ns = scrape_codes(codes_ready_for_update)
    update_post_storage(ns)
else:
    print("No posts to update.")

No posts to update.


In [34]:
# exctract comments and tags in comments for posts in the storage. Add as two attributes in the_posts.
# Also update caption tags from caption
for code in the_posts:
    
    the_posts[code]["caption_tags"] = extract_hashtags(the_posts[code].get("caption",""))
    
    comments = []
    for a in the_posts[code]["activity"]:
        for c in a["comments"]["nodes"]:
            comments += [c['text']]
    comments = list(set(comments))
    the_posts[code]["comm_texts"]=comments
    the_posts[code]["comm_tags"]=extract_hashtags(" ".join(comments))

In [35]:
# download missing media objects
cc = 0
for code in the_posts:
    cc += 1
    if cc % 50 == 0: print()
    download_object(code)

_.____________________.___.________.__.______.___
__.._.____._________._.__________.________________
_____.__._.______.________________________________
_____________________._.._________.__.____________
___._____________________________.____________.___
______________________._______________.___________
._______.__________.______________________________
_________.___________.____________________.__._.__
______

# Dump

In [36]:
f = open(data_path+"/metadata.json","w")
json.dump(the_posts,f)
f.close()

In [37]:
driver.close()

# .

# Aggregate tags and labels into dataframes

The cells generate the following dataframes

s_comm: all comments tags in all posts  
s_cap: all caption tags in all posts  
s_combined: all tags in all posts  
s_labels: all labels (Google) in all posts  

In [38]:
# initialise 
all_labels = []
comm_tags = []
cap_tags = []

In [39]:
for code in the_posts:
    cap_tags += the_posts[code].get("caption_tags",[])
    comm_tags += the_posts[code].get("comm_tags",[]) #include tags found in comments

In [40]:
print("Unique comment tags:",len(set(comm_tags)))
print("Unique caption tags:",len(set(cap_tags)))
print("Tags both in comments and captions:",len(set(comm_tags).intersection(set(cap_tags))))

Unique comment tags: 667
Unique caption tags: 2560
Tags both in comments and captions: 232


In [41]:
s_comm = pd.DataFrame(Counter(comm_tags).most_common(100)[1:],columns=["tags","freq"])
s_comm["type"]="comment"
s_cap = pd.DataFrame(Counter(cap_tags).most_common(100)[1:],columns=["tags","freq"])
s_cap["type"]="caption"
s_combined = s_comm.append(s_cap)
s_combined.index = range(0,len(s_combined))

In [42]:
for code in the_posts:
    if "labels" in the_posts[code]:
        all_labels += list(map(lambda x:x["description"], the_posts[code]["labels"]))

In [43]:
s_labels = pd.DataFrame(Counter(all_labels).most_common(40),columns=["labels","freq"])

In [45]:
s_comm.to_csv(data_path+"/s_comm.csv")
s_cap.to_csv(data_path+"/s_cap.csv")
s_combined.to_csv(data_path+"/s_combined.csv")
s_labels.to_csv(data_path+"/s_labels.csv")

# Create HTML for the Photo Grid 

In [ ]:
def get_total_likes(p):
    likes = 0
    for u in p['activity']:
        if u["video_views"] == "not_a_video":
            likes += u["likes"]["count"]
        else:
            likes += u["video_views"] 
    return likes

In [ ]:
if deets["display"]["selected_labels"]==[]:
    selected_labels = set(all_labels)
else:
    selected_labels = set(deets["display"]["selected_labels"])
    
banned_labels = set(deets["display"]["banned_labels"])

In [ ]:
if deets["display"]["selected_tags"]==[]:
    selected_tags = set(cap_tags+comm_tags)
else:
    selected_tags = set(deets["display"]["selected_tags"])
    
banned_tags = set(deets["display"]["banned_tags"])

In [ ]:
print(len(the_posts),"posts in total")
the_code = ''

cc = 1
the_size = 150
include_this = True


for code in the_posts:
    include_this = True
    if cc > 50000: break
        
    ##############################################################################
    ## Start of the section where photos/videos are selected to be included or not
    if 'labels' in the_posts[code]:
        photo_labels = set(map(lambda x:x["description"], the_posts[code]["labels"]))
    else:
        photo_labels = set()
    
    if  not the_posts[code]["is_video"] and len(all_labels)>0:
        if len(photo_labels.intersection(selected_labels))==0 or \
        len(photo_labels.intersection(banned_labels))>0: 
            include_this = False

    photo_tags =  set(the_posts[code]["caption_tags"]+the_posts[code]["caption_tags"])
    if len(cap_tags+comm_tags)>0:
        if len(photo_tags.intersection(selected_tags))==0 or \
        len(photo_tags.intersection(banned_tags))>0:
            include_this = False

    if get_total_likes(the_posts[code])<deets["display"]["min_likes"] or \
    get_total_likes(the_posts[code])>deets["display"]["max_likes"]:
        include_this = False
    
    if the_posts[code]["is_ad"]: include_this = deets["display"]["include_ads"]=="yes" and include_this
    
    if deets["display"]["include_videos"]=="none" and the_posts[code]["is_video"]: include_this = False
    if deets["display"]["include_photos"]=="none" and not the_posts[code]["is_video"]: include_this = False    
    
    # the conditions below can turn back a "False" generated in the previous statements
    if len(photo_labels)==0 and not the_posts[code]["is_video"]: include_this = True
    if len(photo_tags)==0: include_this = True
    
    if deets["display"]["include_videos"]=="all" and the_posts[code]["is_video"]: include_this = True
    if deets["display"]["include_photos"]=="all" and not the_posts[code]["is_video"]: include_this = True
    ##################################################################################
    ## End of the section for selecting photos/videos
    ##################################################################################
    
    
    if include_this:
        cc += 1
        if not the_posts[code]["is_video"]:
        
            the_code += '<div class="imgWrap" width='+str(the_size)+' height='+str(the_size)+'>'
            the_code += '<a href="https://www.instagram.com/p/'+code+'" target="_blank">'
            the_code += '<img class="centered-and-cropped" src="../media/'+the_posts[code]['the_fname']
            the_code += '" width='+str(the_size)+' height='+str(the_size)+' alt="">'
            the_code += '<p class="imgDescription">'+clstr(the_posts[code].get("caption","")[:300])+'</p>'
            the_code += '</a>'
            the_code += '</div>&nbsp;'
        
        else:
        
            the_code += '<a href="https://www.instagram.com/p/'+code+'" target="_blank">'
            the_code += '<video height="'+str(the_size)
            the_code += '" onmouseover="this.play();" onmouseout="this.pause();">'
            the_code += '<source src="../media/'+the_posts[code]['the_fname']+'" type="video/mp4">'
            the_code += '</video>&nbsp;'
            the_code += '</a>'

print("Ready to display",cc-1,"objects")

In [ ]:
f = open(data_path+"/"+the_item+"_grid.html","w")
f.write('<html><head><link rel="stylesheet" href="../insta.css"></head><body>')
f.write(the_code)
f.write("</body></html>")
f.close()